In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [ ]:
from PIL import Image
import os

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input

In [ ]:
idg=ImageDataGenerator(validation_split=.1,horizontal_flip=True, 
                       brightness_range= [0.2,0.4],
                       preprocessing_function=preprocess_input)

In [ ]:
labels=pd.read_csv('../input/dog-breed-identification/labels.csv')#imports labels
labels.head()

In [ ]:
labels['id']=labels['id']+'.jpg'
labels.head() #this is just to match our id column with original image names

In [ ]:
train_generator=idg.flow_from_dataframe(labels,
                                       '../input/dog-breed-identification/train/',
                                       x_col='id',
                                       y_col='breed',
                                       target_size=(224,224),
                                       batch_size=32,
                                       subset='training')
val_generator=idg.flow_from_dataframe(labels,
                                       '../input/dog-breed-identification/train/',
                                       x_col='id',
                                       y_col='breed',
                                       target_size=(224,224),
                                       batch_size=32,
                                       subset='validation')

In [ ]:
vgg_model=VGG16(include_top=False,input_shape=(224,224,3))#include_top=F, Removes top dense layers
vgg_model.summary()

In [ ]:
#switch off weights training for the conv layers
#add our own layers

In [ ]:
for layer in vgg_model.layers:
    layer.trainable=False

In [ ]:
#one more thing
vgg_model.summary()

In [ ]:
for layer in vgg_model.layers:
    print(f'layer={layer.name} is_trainable : {layer.trainable}')

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
vgg_output=vgg_model.output
# Funtional way of adding layers in Keras
flattened_output=Flatten()(vgg_output)
dense_output=Dense(512,activation=relu)(flattened_output)
dense_output2=Dense(120,activation=softmax)(dense_output)  #Added our own layers


In [ ]:
final_model=Model(inputs=vgg_model.input,outputs=dense_output2)

In [ ]:
final_model.summary()

In [ ]:
for layer in final_model.layers:
    print(f'layer={layer.name} is_trainable : {layer.trainable}')

In [ ]:
final_model.compile(optimizer=SGD(),
    loss=categorical_crossentropy,
    metrics=['acc'])

#one hot encoded classes -->categorical_crossentropy

In [ ]:
! ps -aux | grep tensorboard
# ! kill 2925  

In [ ]:
# TNESORBOARD HAS BEEN SUSPENDED FROM KAGGLE | READ PRODUCT FEEDBACK ON KAGGLE
# from tensorflow.keras.callbacks import TensorBoard
# %load_ext tensorboard
# # %reload_ext tensorboard
# %tensorboard --logdir logs 

In [ ]:
# WE WILL USE A REVERSE PROXY TUNNEL ngrock for our TENSORBOARD
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
import multiprocessing


pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir ./logs/ --host 0.0.0.0 --port 6010 &",
                        "./ngrok http 6010 &"
                        ]]

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
tfb=TensorBoard("logs")

In [ ]:
final_model_history=final_model.fit_generator(train_generator,
                         epochs=30,
                         validation_data=val_generator,callbacks=[tfb])